# 1. vkitti file split
- cycleGAN 모델학습에 필요한 vkitti 이미지 데이터셋을 불러옵니다.
- vkitti 데이터셋으로는 맑은 날씨인 kitti의 clone을 사용하였습니다.

In [3]:
import os
import shutil
import json

vkitti_raw_path = r'C:\Users\junho\Desktop\cycleGAN\vkitti_2.0.3_rgb'
scene_list = os.listdir(vkitti_raw_path)
detail_path = r'\clone\frames\rgb\testA'

In [6]:
img_paths = []  # scene 당 이미지 경로들의 모음
for scene in scene_list:
    # rawpath + scene + img_path => clone의 scene 당 이미지 디렉토리의 경로
    img_path = vkitti_raw_path+'/'+scene+detail_path
    img_list = os.listdir(img_path)  # 이미지 이름 저장
    img_list.sort()
    for i, img in enumerate(img_list):  # 이미지 디렉토리 경로 + 이미지 이름 => 최종 이미지 경로
        img_list[i] = img_path + '\\'+ img
    img_paths.append(img_list) #img_paths[0] = scene 0

In [ ]:
for scene_num, img_path in enumerate(img_paths):  # scene for문
    space_num = int(len(img_path)/100)
    for img_num, img in enumerate(img_path[::space_num]):  # 이미지 개별 for문
        if img_num == 100:
            break
        src = img
        dst = (r'C:\Users\junho\Desktop\cycleGAN\pytorch-CycleGAN-and-pix2pix\datasets\vkitti_final' +
            '\\'+str(scene_num)+str(img_num)+'.jpg')
        shutil.copy(src, dst)

# 2. wheather file split
- bdd100k데이터셋에서 cycleGAN학습에 필요한 시간대/환경별로 이미지 분리시켜줍니다.
- 분리한 이미지들을 심볼릭 링크로 생성합니다.

In [ ]:
#directory 생성
def createDirectory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
#dir format : windows
train_label_path = r'C:\Users\junho\Desktop\cycleGAN\bdd100k\labels\bdd100k_labels_images_train.json'

#label open
with open(train_label_path) as f:
    train_json = json.load(f)

In [ ]:
raw_bdd_train_path = r'C:\Users\junho\Desktop\cycleGAN\bdd100k\images\100k\train'
classfication_path = r'C:\Users\junho\Desktop\cycleGAN\bdd_classification'

#daytime , dawm/dusk
#'rainy','clear','overcast','partly cloudy','foggy'
type_of_time = ['daytime']  # , 'dawn/dusk'
type_of_weather = ['rainy', 'snowy', 'clear',
                   'overcast', 'undefined', 'partly cloudy', 'foggy']

In [ ]:
type_of_img = {}
for weather in type_of_weather :
    type_of_img[weather] =[]

for num, img_data in enumerate(train_json) :
    for time in type_of_time : 
        if time==img_data['attributes']['timeofday'] :
            type_of_img[img_data['attributes']['weather']].append(num)

for key in type_of_img.keys():
    createDirectory(classfication_path+'\\'+key+'\\')

In [ ]:
#syimbolic linked
for type in type_of_img.keys() :
    for img_num in type_of_img[type] :
        src = raw_bdd_train_path+'\\'+train_json[img_num]['name']
        dst = classfication_path+'\\'+type+'\\'+train_json[img_num]['name']
        os.symlink(src, dst)

# 3. random file split
- 시간대/환경별 분리한 bdd100k데이터셋의 크기가 너무 커 일부분만을 사용하였습니다.
- cycleGAN에 학습시킬 데이터셋으로 환경별 500장씩 시드값을 주어 선택하였습니다.

In [ ]:
tmp = os.listdir(
    r'C:\Users\junho\Desktop\cycleGAN\bdd_type_of_time_dataset\daytime')
tmp2 = os.listdir(
    r'C:\Users\junho\Desktop\cycleGAN\bdd_type_of_time_dataset\night')
    
print(len(tmp), len(tmp2))

31728 22971


In [ ]:
import json

#label open
train_label_path = r'C:\Users\junho\Desktop\cycleGAN\bdd100k\labels\bdd100k_labels_images_train.json'
with open(train_label_path) as f :
    train_json = json.load(f)

#train_json[0]

#시간 카테고리 별로 분류, 값은 json list의 enumerate한 값 (이미지 number)
type_of_time= ['daytime','night','dawn/dusk','undefined']
type_of_img ={}

for time in type_of_time :
    type_of_img[time] =[]


for num, img_data in enumerate( train_json) :
    time =img_data['attributes']['timeofday']
    type_of_img[time].append(num)


#분류할 디렉토리 만들기 
def createDirectory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

raw_bdd_train_path = r'C:\Users\junho\Desktop\cycleGAN\bdd100k\images\100k\train\\'
classfication_path = r'C:\Users\junho\Desktop\cycleGAN\bdd_day_night\\'
for time in type_of_time :
    createDirectory(classfication_path+time+'\\')


#syimbolic linked
for type in type_of_img :
    for img_num in type_of_img[type] :
        src = raw_bdd_train_path+train_json[img_num]['name']
        dst = classfication_path+type+'\\'+train_json[img_num]['name']
        os.symlink(src, dst)


In [ ]:
import random 
#36728 27971

random.seed(2022) #시드값
#분리할 파일 경로
classfication_path = r'C:\Users\junho\Desktop\cycleGAN\bdd_type_of_time_dataset'
img_num =500 #이미지 개수
day_random=[] #낮 이미지 리스트
night_random= [] #밤 이미지 리스트

while True :# 중복안되게 5000장의 이미지에 대한 숫자
    day_random.append(random.randint(0,36728))
    if len(day_random)==img_num :
        set_rand = set(day_random)
        if len(set_rand) == img_num :
            break
        else :
            day_random=list(set_rand)


In [ ]:
print(len(day_random))

500


In [ ]:
while True :# 밤의 경우
    night_random.append(random.randint(0,27971))
    if len(night_random)==img_num :
        set_rand = set(night_random)
        if len(set_rand) == img_num :
            break
        else :
            night_random=list(set_rand)

In [ ]:
#심볼릭 링크 파일에 대한 경로들을 정렬
daytime_path = r'C:\Users\junho\Desktop\cycleGAN\bdd_day_night\daytime'
daytime_list = os.listdir(daytime_path)
daytime_list.sort()
night_path = r'C:\Users\junho\Desktop\cycleGAN\bdd_day_night\night'
night_list= os.listdir(night_path)
night_list.sort()

In [ ]:
import shutil
daytime_dst = r'C:\Users\junho\Desktop\cycleGAN\pytorch-CycleGAN-and-pix2pix\datasets\bdd_day_night\trainA'
nighttime_dst = r'C:\Users\junho\Desktop\cycleGAN\pytorch-CycleGAN-and-pix2pix\datasets\bdd_day_night\trainB'
 
 
for daytime,night in zip(day_random,night_random):
    shutil.copy(daytime_path+ '\\'+daytime_list[daytime], daytime_dst)
    shutil.copy(night_path+'\\'+ night_list[night], nighttime_dst)

In [ ]:
len(os.listdir(daytime_dst))

500